In [1]:
import cv2
import numpy as np
import glob
import ultralytics
import supervision as sv
from ultralytics import YOLO
import torch
import datetime
import time
import os
import torchvision
from tqdm import tqdm

import requests
import json
import telebot

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
ultralytics.__version__

'8.1.24'

In [2]:
img = cv2.imread('DATA/Drone/0/seq3-drone_0000001.jpg')
img.shape

(1080, 1920, 3)

In [123]:
BOT_TOKEN = "7024941864:AAE7uWV_FCYFu16uhgOloMPLhyLf7va3rU8"

bot = telebot.TeleBot(BOT_TOKEN)

In [160]:
COLORS = sv.ColorPalette.default()

class VideoProcessor:
    def __init__(
        self,
        source_weights_path: str,
        source_video_path: str,
        save_name: str,
        target_video_path: str = None,
        confidence_threshold: float = 0.3,
        iou_threshold: float = 0.7    
    ) -> None:
        self.source_video_path = source_video_path
        self.target_video_path = target_video_path
        self.confidence_threshold = confidence_threshold
        self.iou_threshold = iou_threshold
        self.save_name = save_name
        
        self.model = YOLO(source_weights_path)
        self.tracker = sv.ByteTrack()
        self.box_annotator = sv.BoxAnnotator(color=COLORS) 
        self.detection_list = []
    
    def process_video(self):
        frame_generator = sv.get_video_frames_generator(source_path=self.source_video_path)
        video_info = sv.VideoInfo.from_video_path(self.source_video_path)
        
        writer = cv2.VideoWriter(self.save_name,  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, video_info.resolution_wh)
        
        for frame in frame_generator:
            processed_frame = self.process_frame(frame)
            writer.write(processed_frame)
            cv2.imshow("frame", processed_frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        bot.send_photo(324192159, telebot.types.InputFile('detections/detected.jpg'), 'ДТП по вул. Личаківська/Смольського')
        
        writer.release()
        cv2.destroyAllWindows
    
    def annotate_frame(
        self,
        frame: np.ndarray,
        detections: sv.Detections
    ) -> np.ndarray:
        
        annotated_frame = frame.copy()
        labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]
        annotated_frame = self.box_annotator.annotate(
            scene=annotated_frame, 
            detections=detections,
            labels=labels
        )
        
        return annotated_frame
    
    def process_frame(self, frame: np.ndarray) -> np.ndarray:
        result = self.model(
            frame,
            verbose=False,
            conf=self.confidence_threshold,
            iou=self.iou_threshold
        )[0]

        detections = sv.Detections.from_ultralytics(result)
        detections = self.tracker.update_with_detections(detections)

        detection = result.tojson()

        self.detection_list.append(detection[1:-1])
        if len(self.detection_list) > 4:
            cv2.imwrite('detections/detected.jpg',frame)
        if detection == '[]':
            self.detection_list = []
        
        return self.annotate_frame(frame,detections)

C:\Users\stepa\AppData\Local\Temp\ipykernel_5768\4187216398.py:1: DeprecationWarning: default is deprecated: `ColorPalette.default()` is deprecated and will be removed in `supervision-0.20.0`. Use `Color.DEFAULT` instead.
  COLORS = sv.ColorPalette.default()


In [162]:
processor = VideoProcessor(
    #source_weights_path="yolov8m.pt",
    source_weights_path="best_model/train13/weights/best.pt",
    source_video_path="DATA/lviv_acc6.MP4",
    confidence_threshold=0.7,
    iou_threshold=0.3,
    save_name='new_detection.mp4'
)

processor.process_video()

In [142]:
# BOT_TOKEN = "7024941864:AAE7uWV_FCYFu16uhgOloMPLhyLf7va3rU8"

# bot = telebot.TeleBot(BOT_TOKEN)

# @bot.message_handler(commands=['start'])
# def send_welcome(message):
#     bot.reply_to(message, "Вітаю, даний бот створений для надсилання інформації про ДТП")

# @bot.message_handler(commands=['hello'])
# def send_hello(message):
#     bot.reply_to(message, "Очікуйте на інформацію про нові випадки ДТП")

# bot.send_img(324192159, img, caption="New image")
    
# bot.infinity_polling()